In [1]:
!pip install keras==2.8

  Using cached keras-2.8.0-py2.py3-none-any.whl (1.4 MB)
  Attempting uninstall: keras
    Found existing installation: Keras 2.3.1
    Uninstalling Keras-2.3.1:
      Successfully uninstalled Keras-2.3.1


In [2]:
import tensorflow as tf
tf.__version__

'2.8.0'

In [3]:
from tensorflow import keras 
keras.__version__
import sklearn

In [4]:
import h5py
import numpy as np
import pandas as pd
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
background = pd.HDFStore('background.hdf5', mode = 'r')
signal = pd.HDFStore('higgs.hdf5', mode = 'r')

In [5]:
backgroundVar = background['df'][['mJJ','cosThetaC','dRB1J1', 'cenPhJJ', 'pTJJ', 'pTBal', 'dEtaJJ', 'dRB1Ph', 'dPhiBBJJ', 'nJets', 'zep','etaJ5']]
higgsVar = signal['df'][['mJJ','cosThetaC','dRB1J1', 'cenPhJJ', 'pTJJ', 'pTBal', 'dEtaJJ', 'dRB1Ph', 'dPhiBBJJ', 'nJets', 'zep','etaJ5']]

In [6]:
backgroundVar['isBackground'] = 1
higgsVar['isBackground'] = 0

In [7]:
backgroundValid, backgroundTrain = backgroundVar[:500000], backgroundVar[500000:] 
higgsValid, higgsTrain = higgsVar[:500000], higgsVar[500000:]
backgroundTrain = backgroundTrain[:500000]
higgsTrain = higgsTrain[:500000]

In [8]:
targetbackground = backgroundTrain['isBackground']
targetHiggs = higgsTrain['isBackground']
target = pd.concat([targetHiggs,targetbackground])
training = pd.concat([higgsTrain,backgroundTrain])
del training['isBackground'];
print(target.shape)
print(training.shape)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(training, target, test_size=0.3) # 70% training and 30% test

(1000000,)
(1000000, 12)


In [9]:
from sklearn import neighbors, datasets
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
model = make_pipeline(StandardScaler(),neighbors.KNeighborsClassifier(n_neighbors=7))
##model = make_pipeline(StandardScaler(),SVC(gamma='auto'))
print(model)
np.random.seed(42)
tf.random.set_seed(42)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('kneighborsclassifier', KNeighborsClassifier(n_neighbors=7))])


In [10]:
model.fit(X_train,y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('kneighborsclassifier', KNeighborsClassifier(n_neighbors=7))])

In [11]:
y_pred = model.predict(X_test)

In [12]:
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.6669533333333333


In [13]:
model = make_pipeline(StandardScaler(),neighbors.KNeighborsClassifier(n_neighbors=9))
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))


Accuracy: 0.6728833333333334


In [16]:
model = make_pipeline(StandardScaler(),SGDClassifier())
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.6671033333333334
